In [2]:
import torch
from torch import nn
from torch.nn import init
import torch.utils.data as data_utils
from torch.autograd import Variable
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
SEED = 2019
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

In [3]:
dataset = np.loadtxt('../ml-1m/ratings.dat', delimiter='::', usecols=[0,1,3], dtype=int)
N_USER = np.max(dataset[:,0])
N_ITEM = np.max(dataset[:,1])

In [4]:
def generate_train_from_local(path, n_user, n_item):
    data = np.loadtxt(fname=path, delimiter="\t", skiprows=1, dtype=int)
    train_matrix = np.zeros((n_user, n_item), dtype = np.int8)
    for line in data:
        train_matrix[line[0],line[1]] = 1
    return train_matrix

def generate_test_from_local(path):
    data = np.loadtxt(fname=path, delimiter="\t", skiprows=1, dtype=int)
    return data

train_matrix = generate_train_from_local(path="../ml-1m/ml.train.txt",n_user=N_USER, n_item=N_ITEM)
test = generate_test_from_local(path="../ml-1m/ml.test.txt")

In [5]:
def generate_train_batch(interact_matrix, batch_size, neg_num=1, shuffle = True, drop_last = False):
    """
    构造训练用的三元组
    对于随机抽出的用户u，i可以从user_ratings随机抽出，而j也是从总的电影集中随机抽出，当然j必须保证(u,j)不在user_ratings中
    """
    n_users = interact_matrix.shape[0]
    n_items = interact_matrix.shape[1]
    one_epoch = list()   # 当前epoch
    index = np.array(range(n_users))
    if shuffle:
        np.random.shuffle(index)  # 用户索引shuffle
    for start_index in range(0, n_users, batch_size):
        if (start_index+batch_size) > n_users and drop_last:  # 不保留最后一个不满batch_size长度的batch
            continue
        end_index = n_users if (start_index+batch_size) > n_users else (start_index+batch_size)
        users = index[start_index: end_index]   # 当前batch的用户编号
        temp = interact_matrix[users]           # 当前batch用户的所有正负交互
        one_batch = list()  # 当前batch
        for u, line in enumerate(temp):
            P = np.nonzero(line)[0]      # 该用户users[u]的所有正例
            N = np.nonzero(line-1)[0]    # 该用户users[u]的所有负例
            i = np.random.choice(P, 1)[0]
            neg_item = np.random.choice(N, neg_num, replace=False)
            for j in neg_item:
                one_batch.append([users[u],i,j])
        one_epoch.append(np.array(one_batch))
    return np.array(one_epoch)

In [6]:
class ConvNCF(nn.Module):
    def __init__(self, fm_sizes, n_user, n_item, n_fm, drop_out, myStride=2, n_output=1):
        ''' e.g.--> fm_sizes = [64,32,16,8,4,2,1] '''
        super(ConvNCF, self).__init__()
        self.convs = list()
        self.dropout = nn.Dropout(p=drop_out)
        self.user_embedding_layer = nn.Embedding(n_user, fm_sizes[0])
        self._set_normalInit(self.user_embedding_layer, hasBias = False) 
        #self._set_xavierInit(self.user_embedding_layer, hasBias = False)
        #self._set_heInit(self.user_embedding_layer, hasBias = False) 
        self.item_embedding_layer = nn.Embedding(n_item, fm_sizes[0])
        self._set_normalInit(self.item_embedding_layer, hasBias = False) 
        #self._set_xavierInit(self.item_embedding_layer, hasBias = False)
        #self._set_heInit(self.item_embedding_layer, hasBias = False) 
        for i in range(1, len(fm_sizes)):
            inChannel = 1 if i == 1 else n_fm
            #conv = nn.Conv2d(in_channels=inChannel, out_channels=32, kernel_size=fm_sizes[i]+myStride, stride=myStride)
            conv = nn.Conv2d(in_channels=inChannel, out_channels=n_fm, kernel_size=4, stride=myStride, padding=1)
            #self._set_normalInit(conv)
            #self._set_xavierInit(conv)
            self._set_heInit(conv)
            setattr(self, 'conv%i' % i, conv)
            self.convs.append(conv)

        self.predict = nn.Linear(n_fm, n_output)         # output layer
        self._set_xavierInit(self.predict)            # parameters initialization
        return
    
    def _set_xavierInit(self, layer, hasBias = True):
        init.xavier_uniform_(layer.weight)
        if hasBias:
            init.constant_(layer.bias, 0.01)
        return
    
    def _set_heInit(self, layer, hasBias = True):
        init.kaiming_normal_(layer.weight, nonlinearity='relu')
        if hasBias:
            init.constant_(layer.bias, 0.01)
        return
    
    def _set_normalInit(self, layer, parameter = [0.0, 0.1], hasBias = True):
        init.normal_(layer.weight, mean = parameter[0], std = parameter[1])
        if hasBias:
            init.constant_(layer.bias, 0.01)
        return
    
    def _set_uniformInit(self, layer, parameter = 1, hasBias = True):
        init.uniform_(layer.weight, a = 0, b = parameter)
        if hasBias:
            init.uniform_(layer.bias, a = 0, b = parameter)
        return
    
    def forward(self, user, item_pos, item_neg, train = True):
        user = self.user_embedding_layer(user)
        item_pos = self.item_embedding_layer(item_pos)
        if train:
            item_neg = self.item_embedding_layer(item_neg)
        x1, x2 = None, None
        temp1, temp2 = list(), list() 
        out1, out2 = None, None
        for i in range(user.size()[0]):
            temp1.append(torch.mm(user[i].T, item_pos[i]))
            if train:
                temp2.append(torch.mm(user[i].T, item_neg[i]))
        x1 = torch.stack(temp1)
        x1 = x1.view(x1.size()[0], -1, x1.size()[1], x1.size()[2])
        if train:
            x2 = torch.stack(temp2)
            x2 = x2.view(x2.size()[0], -1, x2.size()[1], x2.size()[2])
        ''' ## conv2d -input  (batch_size, channel, weight, height) '''
        for conv in self.convs:
            x1 = torch.relu(conv(x1))
            if train:
                x2 = torch.relu(conv(x2))
        ''' ## conv2d -output (batch_size, out_channel, out_weight, out_height) '''
        x1 = torch.flatten(x1, start_dim = 1)
        x1 = self.dropout(x1)
        if train:
            x2 = torch.flatten(x2, start_dim = 1)
            x2 = self.dropout(x2)
        #out1 = torch.sigmoid(self.dropout(self.predict(x1)))
        out1 = torch.sigmoid(self.predict(x1))
        if train:
            #out2 = torch.sigmoid(self.dropout(self.predict(x2)))
            out2 = torch.sigmoid(self.predict(x2))
        return out1, out2

In [7]:
class My_loss(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, y1, y2):
        return torch.sum(torch.log(1+torch.exp(-(y1 - y2))))

In [8]:
def getHitRatio(ranklist, gtItem):
    #HR击中率，如果topk中有正例ID即认为正确
    if gtItem in ranklist:
        return 1
    return 0

def getNDCG(ranklist, gtItem):
    #NDCG归一化折损累计增益
    for i in range(len(ranklist)):
        item = ranklist[i]
        if item == gtItem:
            return np.log(2) / np.log(i+2)
    return 0

In [9]:
def movieEval_1(model, loss_func, test, train_matrix, n_user, n_item, topK = 100):   
    item_list = np.array(range(n_item))
    item_list = torch.from_numpy(item_list.reshape(-1, 1)).type(torch.LongTensor)
    if torch.cuda.is_available():
        item_list = item_list.cuda()
    hit_list = list()
    undcg_list = list()
    rank_all_users = list()
    model.eval()
    with torch.no_grad(): 
        for line in test:
            user = line[0]
            pos_item = line[1]
            user_list = np.array([user for i in range(n_item)])
            user_list = torch.from_numpy(user_list.reshape(-1, 1)).type(torch.LongTensor)
            if torch.cuda.is_available():
                user_list = user_list.cuda()
            prediction, _ = model(user_list, item_list, None, train = False)
            pred_vector = -1 * (prediction.cpu().data.numpy().reshape(-1))
            ranklist = np.argsort(pred_vector)
            real_r = list()
            i = 0
            while len(real_r) < topK:
                if train_matrix[user][ranklist[i]] == 0:
                    real_r.append(ranklist[i])
                i += 1     
            rank_all_users.append(real_r)
            hit_list.append(getHitRatio(real_r, pos_item))
            undcg_list.append(getNDCG(real_r, pos_item))
    model.train()
    hr = np.mean(hit_list)
    ndcg = np.mean(undcg_list)
    print('HR@', topK, ' = %.4f' %  hr)
    print('NDCG@', topK, ' = %.4f' % ndcg)
    return hr, ndcg, rank_all_users

In [10]:
def createModel(fm_sizes, n_fm, lr, drop_out, n_user, n_item):
    model = ConvNCF(fm_sizes=fm_sizes, n_fm=n_fm, drop_out=drop_out, n_user=n_user, n_item=n_item)
    loss_func = My_loss()
    if(torch.cuda.is_available()):
        model = model.cuda()
        loss_func = loss_func.cuda()
    optimizer = torch.optim.Adam(model.parameters(), lr = lr, weight_decay=0.001)
    print(model)
    return model, loss_func, optimizer

In [10]:
def train(train_matrix, test, neg_num, epoch, batch_size, fm_sizes, n_fm, lr, drop_out, n_user, n_item, topK):
    model, loss_func, optimizer = createModel(fm_sizes, n_fm, lr, drop_out, n_user, n_item)
    train_loss_list = list()
    hr_list = [.0]
    ndcg_list = [.0]
    for e in range(epoch):
        data = generate_train_batch(train_matrix, batch_size, neg_num)
        train_loss = list()
        for train_batch in data:
            u = torch.from_numpy(train_batch[:,0].reshape(-1, 1)).type(torch.LongTensor)
            i = torch.from_numpy(train_batch[:,1].reshape(-1, 1)).type(torch.LongTensor)
            j = torch.from_numpy(train_batch[:,2].reshape(-1, 1)).type(torch.LongTensor)
            if (torch.cuda.is_available()):
                u, i, j = u.cuda(), i.cuda(), j.cuda()
            optimizer.zero_grad()
            yui, yuj = model(u, i, j)
            loss = loss_func(yui, yuj) 
            loss.backward()  
            train_loss.append(loss.cpu().item())
            optimizer.step()
        print('------第'+str(e+1)+'个epoch------')
        mean_train_loss = np.mean(train_loss)
        print('train_loss:', mean_train_loss)
        train_loss_list.append(mean_train_loss) 
    '''
        if (e+1)%5==0:
            hr, ndcg, rank_all_users = movieEval_1(model, loss_func, test, train_matrix, n_user, n_item, topK)
            hr_list.append(hr)
            ndcg_list.append(ndcg)
    np.savetxt("./evalres/convncf/train_loss_list_"+str(epoch)+"epoch.txt", train_loss_list)    
    np.savetxt("./evalres/convncf/hr_list_"+str(epoch)+"epoch.txt", hr_list)
    np.savetxt("./evalres/convncf/ndcg_list_"+str(epoch)+"epoch.txt", ndcg_list) 
    '''
    movieEval_1(model, loss_func, test, train_matrix, n_user, n_item, topK)
    torch.cuda.empty_cache()
    print('------Finished------')
    return model

FM_SIZE = [64,32,16,8,4,2,1]
N_FM = 32
N_USER = np.max(dataset[:,0])
N_ITEM = np.max(dataset[:,1])
LEARNING_RATE = 0.001
BATCH_SIZE = 128
DROP_OUT = 0.5
NEG_NUM = 4
EPOCH = 200
TOPK = 100
#train(train_matrix, test, neg_num=NEG_NUM, epoch=EPOCH, batch_size=BATCH_SIZE, fm_sizes=FM_SIZE, n_fm=N_FM, lr=LEARNING_RATE, drop_out=DROP_OUT, n_user=N_USER, n_item=N_ITEM, topK=TOPK)
model = train(train_matrix, test, neg_num=NEG_NUM, epoch=80, batch_size=BATCH_SIZE, fm_sizes=FM_SIZE, n_fm=N_FM, lr=LEARNING_RATE, drop_out=DROP_OUT, n_user=N_USER, n_item=N_ITEM, topK=TOPK)

ConvNCF(
  (dropout): Dropout(p=0.5, inplace=False)
  (user_embedding_layer): Embedding(6040, 64)
  (item_embedding_layer): Embedding(3952, 64)
  (conv1): Conv2d(1, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv3): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv4): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv5): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv6): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (predict): Linear(in_features=32, out_features=1, bias=True)
)
------第1个epoch------
train_loss: 348.89106973012287
------第2个epoch------
train_loss: 340.2500733534495
------第3个epoch------
train_loss: 290.7422326405843
------第4个epoch------
train_loss: 276.81707469622296
------第5个epoch------
train_loss: 268.5294523239136
------第6个epoch------
train_loss: 263.9873646100362
------第7个epoch------
t

In [11]:
torch.save(model, "./evalres/model/ConvNCF.pkl")

In [11]:
def train_eval_d(train_matrix, test, neg_num, epoch, batch_size, fm_sizes, n_fm, lr, drop_out, n_user, n_item, topK):
    hr_list = list()
    ndcg_list = list()
    for d in fm_sizes:
        model, loss_func, optimizer = createModel(d, n_fm, lr, drop_out, n_user, n_item)
        model.train()
        for e in range(epoch):
            data = generate_train_batch(train_matrix, batch_size, neg_num)
            train_loss = list()
            for train_batch in data:
                u = torch.from_numpy(train_batch[:,0].reshape(-1, 1)).type(torch.LongTensor)
                i = torch.from_numpy(train_batch[:,1].reshape(-1, 1)).type(torch.LongTensor)
                j = torch.from_numpy(train_batch[:,2].reshape(-1, 1)).type(torch.LongTensor)
                if (torch.cuda.is_available()):
                    u, i, j = u.cuda(), i.cuda(), j.cuda()
                optimizer.zero_grad()
                yui, yuj = model(u, i, j)
                loss = loss_func(yui, yuj) 
                loss.backward() 
                optimizer.step()
        hr, ndcg, rank_all_users = movieEval_1(model, loss_func, test, train_matrix, n_user, n_item, topK)
        hr_list.append(hr)
        ndcg_list.append(ndcg)   
    np.savetxt("./evalres/convncf/hr_list_d.txt", hr_list)
    np.savetxt("./evalres/convncf/ndcg_list_d.txt", ndcg_list) 
    torch.cuda.empty_cache()
    print('------Finished------')
    return

FM_SIZE = [[8,4,2,1],[16,8,4,2,1],[32,16,8,4,2,1],[64,32,16,8,4,2,1]]
N_FM = 32
N_USER = np.max(dataset[:,0])
N_ITEM = np.max(dataset[:,1])
LEARNING_RATE = 0.001
BATCH_SIZE = 128
DROP_OUT = 0.5
NEG_NUM = 4
EPOCH = 60
TOPK = 100
train_eval_d(train_matrix, test, neg_num=NEG_NUM, epoch=EPOCH, batch_size=BATCH_SIZE, fm_sizes=FM_SIZE, n_fm=N_FM, 
             lr=LEARNING_RATE, drop_out=DROP_OUT, n_user=N_USER, n_item=N_ITEM, topK=TOPK)

ConvNCF(
  (dropout): Dropout(p=0.5, inplace=False)
  (user_embedding_layer): Embedding(6040, 8)
  (item_embedding_layer): Embedding(3952, 8)
  (conv1): Conv2d(1, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv3): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (predict): Linear(in_features=32, out_features=1, bias=True)
)
HR@ 100  = 0.2243
NDCG@ 100  = 0.0468
ConvNCF(
  (dropout): Dropout(p=0.5, inplace=False)
  (user_embedding_layer): Embedding(6040, 16)
  (item_embedding_layer): Embedding(3952, 16)
  (conv1): Conv2d(1, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv3): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv4): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (predict): Linear(in_features=32, out_features=1, bias=True)
)
HR@ 100  

In [14]:
def train_eval_topK(train_matrix, test, neg_num, epoch, batch_size, fm_sizes, n_fm, lr, drop_out, n_user, n_item, topK):
    hr_list = list()
    ndcg_list = list()
    model, loss_func, optimizer = createModel(fm_sizes, n_fm, lr, drop_out, n_user, n_item)
    model.train()
    for e in range(epoch):
        data = generate_train_batch(train_matrix, batch_size, neg_num)
        train_loss = list()
        for train_batch in data:
            u = torch.from_numpy(train_batch[:,0].reshape(-1, 1)).type(torch.LongTensor)
            i = torch.from_numpy(train_batch[:,1].reshape(-1, 1)).type(torch.LongTensor)
            j = torch.from_numpy(train_batch[:,2].reshape(-1, 1)).type(torch.LongTensor)
            if (torch.cuda.is_available()):
                u, i, j = u.cuda(), i.cuda(), j.cuda()
            optimizer.zero_grad()
            yui, yuj = model(u, i, j)
            loss = loss_func(yui, yuj) 
            loss.backward() 
            optimizer.step()
    for k in topK:
        hr, ndcg, rank_all_users = movieEval_1(model, loss_func, test, train_matrix, n_user, n_item, k)
        hr_list.append(hr)
        ndcg_list.append(ndcg)   
    np.savetxt("./evalres/convncf/hr_list_topk.txt", hr_list)
    np.savetxt("./evalres/convncf/ndcg_list_topk.txt", ndcg_list) 
    torch.cuda.empty_cache()
    print('------Finished------')
    return

FM_SIZE = [64,32,16,8,4,2,1]
N_FM = 32
N_USER = np.max(dataset[:,0])
N_ITEM = np.max(dataset[:,1])
LEARNING_RATE = 0.001
BATCH_SIZE = 128
DROP_OUT = 0.5
NEG_NUM = 4
EPOCH = 60
TOPK = [50,100,200]
train_eval_topK(train_matrix, test, neg_num=NEG_NUM, epoch=EPOCH, batch_size=BATCH_SIZE, fm_sizes=FM_SIZE, n_fm=N_FM, 
             lr=LEARNING_RATE, drop_out=DROP_OUT, n_user=N_USER, n_item=N_ITEM, topK=TOPK)

ConvNCF(
  (dropout): Dropout(p=0.5, inplace=False)
  (user_embedding_layer): Embedding(6040, 64)
  (item_embedding_layer): Embedding(3952, 64)
  (conv1): Conv2d(1, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv3): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv4): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv5): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv6): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (predict): Linear(in_features=32, out_features=1, bias=True)
)
HR@ 50  = 0.0646
NDCG@ 50  = 0.0149
HR@ 100  = 0.1440
NDCG@ 100  = 0.0278
HR@ 200  = 0.2373
NDCG@ 200  = 0.0407
------Finished------


In [13]:
def train_eval_negNum(train_matrix, test, neg_num, epoch, batch_size, fm_sizes, n_fm, lr, drop_out, n_user, n_item, topK):
    hr_list = list()
    ndcg_list = list()
    for n in neg_num:
        model, loss_func, optimizer = createModel(fm_sizes, n_fm, lr, drop_out, n_user, n_item)
        model.train()
        for e in range(epoch):
            data = generate_train_batch(train_matrix, batch_size, n)
            train_loss = list()
            for train_batch in data:
                u = torch.from_numpy(train_batch[:,0].reshape(-1, 1)).type(torch.LongTensor)
                i = torch.from_numpy(train_batch[:,1].reshape(-1, 1)).type(torch.LongTensor)
                j = torch.from_numpy(train_batch[:,2].reshape(-1, 1)).type(torch.LongTensor)
                if (torch.cuda.is_available()):
                    u, i, j = u.cuda(), i.cuda(), j.cuda()
                optimizer.zero_grad()
                yui, yuj = model(u, i, j)
                loss = loss_func(yui, yuj) 
                loss.backward() 
                optimizer.step()
        hr, ndcg, rank_all_users = movieEval_1(model, loss_func, test, train_matrix, n_user, n_item, topK)
        hr_list.append(hr)
        ndcg_list.append(ndcg)   
    np.savetxt("./evalres/convncf/hr_list_neg.txt", hr_list)
    np.savetxt("./evalres/convncf/ndcg_list_neg.txt", ndcg_list) 
    torch.cuda.empty_cache()
    print('------Finished------')
    return

FM_SIZE = [64,32,16,8,4,2,1]
N_FM = 32
N_USER = np.max(dataset[:,0])
N_ITEM = np.max(dataset[:,1])
LEARNING_RATE = 0.001
BATCH_SIZE = 128
DROP_OUT = 0.5
NEG_NUM = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
EPOCH = 60
TOPK = 100
train_eval_negNum(train_matrix, test, neg_num=NEG_NUM, epoch=EPOCH, batch_size=BATCH_SIZE, fm_sizes=FM_SIZE, n_fm=N_FM, 
             lr=LEARNING_RATE, drop_out=DROP_OUT, n_user=N_USER, n_item=N_ITEM, topK=TOPK)

ConvNCF(
  (dropout): Dropout(p=0.5, inplace=False)
  (user_embedding_layer): Embedding(6040, 64)
  (item_embedding_layer): Embedding(3952, 64)
  (conv1): Conv2d(1, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv3): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv4): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv5): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv6): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (predict): Linear(in_features=32, out_features=1, bias=True)
)
HR@ 100  = 0.0000
NDCG@ 100  = 0.0000
ConvNCF(
  (dropout): Dropout(p=0.5, inplace=False)
  (user_embedding_layer): Embedding(6040, 64)
  (item_embedding_layer): Embedding(3952, 64)
  (conv1): Conv2d(1, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1,

In [10]:
def train_eval_negNum(train_matrix, test, neg_num, epoch, batch_size, fm_sizes, n_fm, lr, drop_out, n_user, n_item, topK):
    hr_list = list()
    ndcg_list = list()
    for n in neg_num:
        model, loss_func, optimizer = createModel(fm_sizes, n_fm, lr, drop_out, n_user, n_item)
        model.train()
        for e in range(epoch):
            data = generate_train_batch(train_matrix, batch_size, n)
            train_loss = list()
            for train_batch in data:
                u = torch.from_numpy(train_batch[:,0].reshape(-1, 1)).type(torch.LongTensor)
                i = torch.from_numpy(train_batch[:,1].reshape(-1, 1)).type(torch.LongTensor)
                j = torch.from_numpy(train_batch[:,2].reshape(-1, 1)).type(torch.LongTensor)
                if (torch.cuda.is_available()):
                    u, i, j = u.cuda(), i.cuda(), j.cuda()
                optimizer.zero_grad()
                yui, yuj = model(u, i, j)
                loss = loss_func(yui, yuj) 
                loss.backward() 
                optimizer.step()
        hr, ndcg, rank_all_users = movieEval_1(model, loss_func, test, train_matrix, n_user, n_item, topK)
        hr_list.append(hr)
        ndcg_list.append(ndcg)   
    np.savetxt("./evalres/convncf/hr_list_neg_1.txt", hr_list)
    np.savetxt("./evalres/convncf/ndcg_list_neg_1.txt", ndcg_list) 
    torch.cuda.empty_cache()
    print('------Finished------')
    return

FM_SIZE = [64,32,16,8,4,2,1]
N_FM = 32
N_USER = np.max(dataset[:,0])
N_ITEM = np.max(dataset[:,1])
LEARNING_RATE = 0.001
BATCH_SIZE = 128
DROP_OUT = 0.5
NEG_NUM = [1]
EPOCH = 60
TOPK = 100
train_eval_negNum(train_matrix, test, neg_num=NEG_NUM, epoch=EPOCH, batch_size=BATCH_SIZE, fm_sizes=FM_SIZE, n_fm=N_FM, 
             lr=LEARNING_RATE, drop_out=DROP_OUT, n_user=N_USER, n_item=N_ITEM, topK=TOPK)

ConvNCF(
  (dropout): Dropout(p=0.5, inplace=False)
  (user_embedding_layer): Embedding(6040, 64)
  (item_embedding_layer): Embedding(3952, 64)
  (conv1): Conv2d(1, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv3): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv4): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv5): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv6): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (predict): Linear(in_features=32, out_features=1, bias=True)
)
HR@ 100  = 0.1364
NDCG@ 100  = 0.0253
------Finished------


In [11]:
def train_eval_drop_out(train_matrix, test, neg_num, epoch, batch_size, fm_sizes, n_fm, lr, drop_out, n_user, n_item, topK):
    hr_list = list()
    ndcg_list = list()
    for dout in drop_out:
        model, loss_func, optimizer = createModel(fm_sizes, n_fm, lr, dout, n_user, n_item)
        model.train()
        for e in range(epoch):
            data = generate_train_batch(train_matrix, batch_size, neg_num)
            train_loss = list()
            for train_batch in data:
                u = torch.from_numpy(train_batch[:,0].reshape(-1, 1)).type(torch.LongTensor)
                i = torch.from_numpy(train_batch[:,1].reshape(-1, 1)).type(torch.LongTensor)
                j = torch.from_numpy(train_batch[:,2].reshape(-1, 1)).type(torch.LongTensor)
                if (torch.cuda.is_available()):
                    u, i, j = u.cuda(), i.cuda(), j.cuda()
                optimizer.zero_grad()
                yui, yuj = model(u, i, j)
                loss = loss_func(yui, yuj) 
                loss.backward() 
                optimizer.step()
        hr, ndcg, rank_all_users = movieEval_1(model, loss_func, test, train_matrix, n_user, n_item, topK)
        hr_list.append(hr)
        ndcg_list.append(ndcg)   
    np.savetxt("./evalres/convncf/hr_list_drop_out.txt", hr_list)
    np.savetxt("./evalres/convncf/ndcg_list_drop_out.txt", ndcg_list) 
    torch.cuda.empty_cache()
    print('------Finished------')
    return

FM_SIZE = [64,32,16,8,4,2,1]
N_FM = 32
N_USER = np.max(dataset[:,0])
N_ITEM = np.max(dataset[:,1])
LEARNING_RATE = 0.001
BATCH_SIZE = 128
DROP_OUT = [0.1, 0.3, 0.5, 0.7, 0.9]
NEG_NUM = 4
EPOCH = 60
TOPK = 100
train_eval_drop_out(train_matrix, test, neg_num=NEG_NUM, epoch=EPOCH, batch_size=BATCH_SIZE, fm_sizes=FM_SIZE, n_fm=N_FM, 
             lr=LEARNING_RATE, drop_out=DROP_OUT, n_user=N_USER, n_item=N_ITEM, topK=TOPK)

ConvNCF(
  (dropout): Dropout(p=0.1, inplace=False)
  (user_embedding_layer): Embedding(6040, 64)
  (item_embedding_layer): Embedding(3952, 64)
  (conv1): Conv2d(1, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv3): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv4): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv5): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv6): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (predict): Linear(in_features=32, out_features=1, bias=True)
)
HR@ 100  = 0.1808
NDCG@ 100  = 0.0435
ConvNCF(
  (dropout): Dropout(p=0.3, inplace=False)
  (user_embedding_layer): Embedding(6040, 64)
  (item_embedding_layer): Embedding(3952, 64)
  (conv1): Conv2d(1, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1,

In [11]:
def train_eval_drop_out(train_matrix, test, neg_num, epoch, batch_size, fm_sizes, n_fm, lr, drop_out, n_user, n_item, topK):
    hr_list = list()
    ndcg_list = list()
    for dout in drop_out:
        model, loss_func, optimizer = createModel(fm_sizes, n_fm, lr, dout, n_user, n_item)
        model.train()
        for e in range(epoch):
            data = generate_train_batch(train_matrix, batch_size, neg_num)
            train_loss = list()
            for train_batch in data:
                u = torch.from_numpy(train_batch[:,0].reshape(-1, 1)).type(torch.LongTensor)
                i = torch.from_numpy(train_batch[:,1].reshape(-1, 1)).type(torch.LongTensor)
                j = torch.from_numpy(train_batch[:,2].reshape(-1, 1)).type(torch.LongTensor)
                if (torch.cuda.is_available()):
                    u, i, j = u.cuda(), i.cuda(), j.cuda()
                optimizer.zero_grad()
                yui, yuj = model(u, i, j)
                loss = loss_func(yui, yuj) 
                loss.backward() 
                optimizer.step()
        hr, ndcg, rank_all_users = movieEval_1(model, loss_func, test, train_matrix, n_user, n_item, topK)
        hr_list.append(hr)
        ndcg_list.append(ndcg)   
    np.savetxt("./evalres/convncf/hr_list_drop_out_0.9.txt", hr_list)
    np.savetxt("./evalres/convncf/ndcg_list_drop_out_0.9.txt", ndcg_list) 
    torch.cuda.empty_cache()
    print('------Finished------')
    return

FM_SIZE = [64,32,16,8,4,2,1]
N_FM = 32
N_USER = np.max(dataset[:,0])
N_ITEM = np.max(dataset[:,1])
LEARNING_RATE = 0.001
BATCH_SIZE = 128
DROP_OUT = [0.9]
NEG_NUM = 4
EPOCH = 60
TOPK = 100
train_eval_drop_out(train_matrix, test, neg_num=NEG_NUM, epoch=EPOCH, batch_size=BATCH_SIZE, fm_sizes=FM_SIZE, n_fm=N_FM, 
             lr=LEARNING_RATE, drop_out=DROP_OUT, n_user=N_USER, n_item=N_ITEM, topK=TOPK)

ConvNCF(
  (dropout): Dropout(p=0.9, inplace=False)
  (user_embedding_layer): Embedding(6040, 64)
  (item_embedding_layer): Embedding(3952, 64)
  (conv1): Conv2d(1, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv3): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv4): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv5): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv6): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (predict): Linear(in_features=32, out_features=1, bias=True)
)
HR@ 100  = 0.2065
NDCG@ 100  = 0.0459
------Finished------


In [18]:
import time
def train_eval_time(train_matrix, test, neg_num, epoch, batch_size, fm_sizes, n_fm, lr, drop_out, n_user, n_item, topK):
    model, loss_func, optimizer = createModel(fm_sizes, n_fm, lr, drop_out, n_user, n_item)
    model.train()
    time_start, time_end = 0, 0
    for e in range(1):
        data = generate_train_batch(train_matrix, batch_size, neg_num)
        time_start=time.time()
        for train_batch in data:
            u = torch.from_numpy(train_batch[:,0].reshape(-1, 1)).type(torch.LongTensor)
            i = torch.from_numpy(train_batch[:,1].reshape(-1, 1)).type(torch.LongTensor)
            j = torch.from_numpy(train_batch[:,2].reshape(-1, 1)).type(torch.LongTensor)
            if (torch.cuda.is_available()):
                u, i, j = u.cuda(), i.cuda(), j.cuda()
            optimizer.zero_grad()
            yui, yuj = model(u, i, j)
            loss = loss_func(yui, yuj) 
            loss.backward() 
            optimizer.step()
        time_end=time.time()
    print('time cost:', time_end-time_start)
    np.savetxt("./evalres/convncf/single_time.txt", [time_end-time_start]) 
    torch.cuda.empty_cache()
    print('------Finished------')
    return

FM_SIZE = [64,32,16,8,4,2,1]
N_FM = 32
N_USER = np.max(dataset[:,0])
N_ITEM = np.max(dataset[:,1])
LEARNING_RATE = 0.001
BATCH_SIZE = 128
DROP_OUT = 0.5
NEG_NUM = 4
EPOCH = 60
TOPK = 100
train_eval_time(train_matrix, test, neg_num=NEG_NUM, epoch=EPOCH, batch_size=BATCH_SIZE, fm_sizes=FM_SIZE, n_fm=N_FM, 
             lr=LEARNING_RATE, drop_out=DROP_OUT, n_user=N_USER, n_item=N_ITEM, topK=TOPK)

ConvNCF(
  (dropout): Dropout(p=0.5, inplace=False)
  (user_embedding_layer): Embedding(6040, 64)
  (item_embedding_layer): Embedding(3952, 64)
  (conv1): Conv2d(1, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv3): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv4): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv5): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv6): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (predict): Linear(in_features=32, out_features=1, bias=True)
)
time cost: 7.253255367279053
------Finished------
